# Working with Edgar datasets: Wrangling, Pre-processing and exploratory data analysis

In [7]:
!pip install requests

    100% |████████████████████████████████| 92kB 1.7MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 3.0MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 7.8MB/s eta 0:00:01
    100% |████████████████████████████████| 143kB 6.5MB/s eta 0:00:01
    100% |████████████████████████████████| 358kB 2.3MB/s eta 0:00:01


In [9]:
!pip install pandas

    100% |████████████████████████████████| 14.7MB 78kB/s  eta 0:00:01    87% |████████████████████████████    | 12.9MB 17.8MB/s eta 0:00:01
    100% |████████████████████████████████| 4.5MB 257kB/s eta 0:00:01
    100% |████████████████████████████████| 491kB 2.2MB/s eta 0:00:01


In [5]:
!pip install lxml

    100% |████████████████████████████████| 8.6MB 133kB/s eta 0:00:01


In [37]:
from lxml import html
from lxml.html import parse
import requests
import html5lib
#from BeautifulSoup import BeautifulSoup
import pandas as pd
from pandas.io.parsers import TextParser
from urllib.request import urlopen

Input variables (need to set as environmental variables for Docker pipline)

In [11]:
cik = '51143'
acc_no = '0000051143-13-000007'
acc_short = acc_no[0:10] + acc_no[11:13] + acc_no[14:]
#print(acc_short)   

Generate full URL and get the page

In [12]:
page = requests.get('https://www.sec.gov/Archives/edgar/data/' + cik + '/' + acc_short + '/' + acc_no +'-index.htm')
source = page.content
#print(source)

Scrape page content for links, find 10Q

In [13]:
htmlcontent = html.document_fromstring(source)
#print(htmlcontent)

In [14]:
href = ''
for elem in htmlcontent.iterlinks():
    #print(elem[2])
    if "10q.htm" in elem[2]:
        href+= elem[2]

In [15]:
url10q = 'https://www.sec.gov' + href
print(url10q)

https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm


In [28]:
parsed = parse(urlopen(url10q))
doc = parsed.getroot()

In [88]:
tables = doc.findall('.//table')
example = tables[6]
#print(example)

In [93]:
def _unpack(row, kind='td'):
    elements = row.findall('.//%s' % kind)
    return [val.text_content() for val in elements]

#rows = example.findall('.//tr')
#for row in rows:
#    print(_unpack(row, kind='td'))
#data = [_unpack(row, kind='td') for row in rows]    
#print(data)

In [96]:
def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    if len(header) > 0:
        data = [_unpack(r) for r in rows[1:]] 
    else: 
        data = [_unpack(r) for r in rows]     
    print(len(data))
    #for r in data:
    #    print(r)
    return TextParser(data, names=header).get_chunk()
    
example_data = parse_options_data(example)
#print(example_data)

35


TypeError: object of type 'NoneType' has no len()